In [209]:
import cobra
import numpy as np
import pandas as pd

In [210]:

model_ez = cobra.io.read_sbml_model('../../models/e_coli/momentiJO1366.xml')
model_ez.id = 'moment-iJO1366'
model_p = cobra.io.read_sbml_model('../../models/iJN1463.xml')
model_p.solver = 'gurobi'

In [211]:
model_ez.reactions[-1]

Reaction identifier,ER_pool_TG_
Name,prot_pool reaction for unmeasured proteins
Memory address,0x37f783410
Stoichiometry,--> prot_pool --> prot_pool pseudometabolite for unmeasured proteins
GPR,
Lower bound,0.0
Upper bound,0.095


In [212]:
prot_coeffs = []
for r in model_ez.metabolites.prot_pool.reactions:
    try:
        coeff = r.metabolites[model_ez.metabolites.prot_pool]
    except KeyError:
        pass
    else:
        prot_coeffs.append(coeff)

In [213]:
prot_median = np.median(prot_coeffs)

In [214]:
model_p.add_metabolites([model_ez.metabolites.prot_pool])
model_p.add_reactions([model_ez.reactions.ER_pool_TG_])

In [215]:
reactions_to_add = []
reactions_to_remove = []
for r in model_p.reactions:
    if len(r.metabolites) <2:
        continue

    if r.lower_bound < 0:
        r_reverse = cobra.Reaction(f'{r.id}_TG_reverse', lower_bound = 0, upper_bound = 1000)
        r_forward = cobra.Reaction(f'{r.id}_TG_forward', lower_bound = 0, upper_bound = 1000)
        r_reverse.add_metabolites({m:-1*c for m,c in r.metabolites.items()})
        r_forward.add_metabolites({m:c for m,c in r.metabolites.items()})
        # r_reverse.bounds = (0, -r.lower_bound)
        # r_forward.bounds = (0, r.upper_bound)
        r_reverse.gene_reaction_rule = r.gene_reaction_rule
        r_forward.gene_reaction_rule = r.gene_reaction_rule
        reactions_to_add += [r_forward, r_reverse]
        reactions_to_remove.append(r)
        
        
        

In [216]:
model_p.add_reactions(reactions_to_add)
len(model_p.reactions)

4450

In [217]:
model_p.remove_reactions(reactions_to_remove)
len(model_p.reactions)

3689

In [218]:
model_p.solver.update()

In [219]:
# for r in model_p.reactions:
#     if r.lower_bound < 0:
#         print(r)

In [220]:
model_ez_reaction_ids = [r.id for r in model_ez.reactions]
prot_pool = model_p.metabolites.prot_pool
for r in model_p.reactions:
    if len(r.metabolites) <2:
        continue
        
    if len(r.genes) and not ('s0001' in r.gene_name_reaction_rule):
        if r.id in model_ez_reaction_ids:
            r_ez = model_ez.reactions.get_by_id(r.id)
            try:
                coeff = r_ez.metabolites[model_ez.metabolites.prot_pool]
            except KeyError:
                coeff = prot_median
        else:
            coeff = prot_median
        if 0:
            # Uniform weights
            coeff = prot_median
        if coeff > 0:
            print(r.id, coeff)
        # print(r.id, coeff)
        r.add_metabolites({prot_pool:coeff})
            

In [221]:
model_p.reactions.ER_pool_TG_.upper_bound = 0.02

In [222]:
solution = model_p.optimize()

In [223]:
model_p.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
prot_pool,ER_pool_TG_,0.02,0,0.00%
ca2_e,EX_ca2_e,0.0005083,0,0.00%
cl_e,EX_cl_e,0.0005083,0,0.00%
cobalt2_e,EX_cobalt2_e,0.0003657,0,0.00%
cu2_e,EX_cu2_e,0.0003388,0,0.00%
fe2_e,EX_fe2_e,0.001767,0,0.00%
glc__D_e,EX_glc__D_e,4.066,6,100.00%
k_e,EX_k_e,0.01907,0,0.00%
mg2_e,EX_mg2_e,0.0008473,0,0.00%
mn2_e,EX_mn2_e,0.0003388,0,0.00%


# Save ez-model for P. putida

In [224]:
cobra.io.write_sbml_model(model_p, '../../models/momentiJN1463.xml')

In [157]:
df = pd.DataFrame(solution.fluxes)

In [158]:
active_reaction_fluxes = df.loc[df.fluxes!=0]

In [167]:
# active_reaction_fluxes.loc[active_reaction_fluxes.fluxes < 0]

In [138]:
active_reaction_fluxes

,fluxes
3HAD160,0.127988
2DHGLCK,6.000000
2DHGLCNkt_tpp,6.000000
3HAD100,0.147759
3HAD120,0.138109
...,...
3OAR60_TG_forward,0.157532
3OAR80_TG_forward,0.157532
3PQQS_TG_forward,0.000124
4HBADH_TG_forward,0.000124


In [166]:
for r_id, flux in active_reaction_fluxes.iterrows():
    r = model_p.reactions.get_by_id(r_id)
    try:
        p_coeff = r.metabolites[model_p.metabolites.prot_pool]
    except KeyError:
        p_coeff = 0
    if p_coeff*flux['fluxes'] > 0:
        print(r_id, flux['fluxes'])
        print(p_coeff*flux['fluxes'])


ER_pool_TG_ 0.095
0.095


In [109]:
r.metabolites

{<Metabolite 4hpro_LT_e at 0x2a8c259d0>: 1.0,
 <Metabolite 4hpro_LT_p at 0x2a8c2dc10>: -1.0}

In [106]:
model_p.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
prot_pool,ER_pool_TG_,1E-06,0,0.00%
ca2_e,EX_ca2_e,0.002477,0,0.00%
cl_e,EX_cl_e,0.002477,0,0.00%
cobalt2_e,EX_cobalt2_e,0.001782,0,0.00%
cu2_e,EX_cu2_e,0.001651,0,0.00%
fe2_e,EX_fe2_e,0.008608,0,0.00%
glc__D_e,EX_glc__D_e,6,6,99.99%
k_e,EX_k_e,0.0929,0,0.00%
mg2_e,EX_mg2_e,0.004129,0,0.00%
mn2_e,EX_mn2_e,0.001651,0,0.00%


In [95]:
model_p.metabolites.glcn_c.summary()

Percent,Flux,Reaction,Definition
100.00%,6,GLCNt2rpp_TG_forward,glcn_p + h_p + 0.0003759404442969329 prot_pool --> glcn_c + h_c
Percent,Flux,Reaction,Definition
100.00%,-6,GNK,atp_c + glcn_c + 0.002310052411347516 prot_pool --> 6pgc_c + adp_c + h_c
